### 1.1. Escrapear datos

Descripcion: Utilizamos los datos reflejados en la web fuente para la creación de dos DataFrame.

#### 1.1.1. Web pagina Inicial Vinos "Sin Alcohol".
Tomamos la lista de 63 tipos de vinos y obtuvimos los principales datos sobre cada uno:
- Nombre
- Variedad
- Precio

#### 1.1.2. Webs secuntarias, entramos a la url por cada vino disponible en la pagina inicial.
Obtuvimos los datos unicos sobre cada uno:
- Volumen
- Variedad
- Graduación
- Productor
- Etiqueta

ESCRAPEO DE DATOS

Fuente: https://www.thebluedolphinstore.com/

### 1.1.1
Paso 1: instalamos selenium e importamos librerias

In [287]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [525]:
import requests as req

from bs4 import BeautifulSoup as bs

import pandas as pd

In [289]:
url = 'https://www.thebluedolphinstore.com/categoria-producto/vinos-sin-alcohol/'

In [290]:
html=req.get(url).text

sopa=bs(html, 'html.parser')


In [291]:
from selenium import webdriver   # driver de selenium

from selenium.webdriver.chrome.options import Options   # opciones del driver

from selenium.webdriver.common.by import By # By es para buscar por tag, clase, id...

from selenium.webdriver.common.keys import Keys  # para usar teclado

In [292]:
PATH = 'driver/chromedriver'

In [293]:
# opciones del driver
opciones=Options()

# quita la bandera de ser robot
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)


# guardar las cookies
opciones.add_argument('user-data-dir=cookies')    # mantiene las coockies

In [294]:
driver = webdriver.Chrome()

driver.get(url) 

 Paso 2: comenzamos la busqueda de datos

In [295]:
productos = driver.find_elements(By.CLASS_NAME, "product-wrap")

len(productos)

63

In [296]:
productos[0].text.split('\n')

['NUEV@', 'Carl Jung Rosé – Vino Rosado Sin Alcohol', '6,86€ IVA inc.']

In [297]:
nombre_variedad = productos[0].find_element(By.TAG_NAME, 'h3').text.split(' – ')

nombre = nombre_variedad[0]


In [298]:
nombre

'Carl Jung Rosé'

In [299]:
variedad = nombre_variedad[1]

variedad

'Vino Rosado Sin Alcohol'

In [300]:
precio = productos[0].find_element(By.CSS_SELECTOR,'#ajax-content-wrap > div.container-wrap > div > div.row > div.post-area.col.span_9.col_last > ul > li.minimal.product.type-product.post-28958.status-publish.first.instock.product_cat-destacados.product_cat-vino-rosado-sin-alcohol.product_cat-vinos-sin-alcohol.product_tag-vegana-o.has-post-thumbnail.taxable.shipping-taxable.purchasable.product-type-simple.yith-wcbm-product-has-badges.hover-bound > div.product-wrap > div > div > span > span > bdi').text

precio

'6,86€'

In [301]:
precio = float(precio.replace('€', '').replace(',', '.'))

precio

6.86

Paso 3: Obtenemos la data y creamos el DataFrame

In [302]:
data = []

for p in productos[0:]:
    
    nombre = p.find_element(By.TAG_NAME, 'h3').text
    
    #precio = p.find_element(By.CSS_SELECTOR,'#ajax-content-wrap > div.container-wrap > div > div.row > div.post-area.col.span_9.col_last > ul > li.minimal.product.type-product.post-28958.status-publish.first.instock.product_cat-destacados.product_cat-vino-rosado-sin-alcohol.product_cat-vinos-sin-alcohol.product_tag-vegana-o.has-post-thumbnail.taxable.shipping-taxable.purchasable.product-type-simple.yith-wcbm-product-has-badges.hover-bound > div.product-wrap > div > div > span > span > bdi').text
    
    
    #precio = float(precio.replace('€', '').replace('\n', '.'))
    
    
    dictio = {'nombre': nombre, 'variedad' : variedad, 'precio': precio}
    
    data.append(dictio)

In [303]:
data

[{'nombre': 'Carl Jung Rosé – Vino Rosado Sin Alcohol',
  'variedad': 'Vino Rosado Sin Alcohol',
  'precio': 6.86},
 {'nombre': 'Schloss Raggendorf Shiraz Rosé – Vino Rosado Sin Alcohol',
  'variedad': 'Vino Rosado Sin Alcohol',
  'precio': 6.86},
 {'nombre': 'Carl Jung Cabernet Sauvignon – Vino Tinto Sin Alcohol',
  'variedad': 'Vino Rosado Sin Alcohol',
  'precio': 6.86},
 {'nombre': 'Carl Jung Merlot – Vino Tinto Sin Alcohol',
  'variedad': 'Vino Rosado Sin Alcohol',
  'precio': 6.86},
 {'nombre': 'Schloss Raggendorf Zweigelt – Vino Tinto Sin Alcohol',
  'variedad': 'Vino Rosado Sin Alcohol',
  'precio': 6.86},
 {'nombre': 'Schloss Raggendorf Cabernet Sauvignon – Vino Tinto Sin Alcohol',
  'variedad': 'Vino Rosado Sin Alcohol',
  'precio': 6.86},
 {'nombre': 'Carl Jung Chardonnay – Vino Blanco Sin Alcohol',
  'variedad': 'Vino Rosado Sin Alcohol',
  'precio': 6.86},
 {'nombre': 'Carl Jung Riesling – Vino Blanco Sin Alcohol',
  'variedad': 'Vino Rosado Sin Alcohol',
  'precio': 6.86}

In [529]:
## primer DataFrame listo:

listado = pd.DataFrame(data)

listado.head()

,nombre,variedad,precio
0,Carl Jung Rosé – Vino Rosado Sin Alcohol,Vino Rosado Sin Alcohol,6.86
1,Schloss Raggendorf Shiraz Rosé – Vino Rosado S...,Vino Rosado Sin Alcohol,6.86
2,Carl Jung Cabernet Sauvignon – Vino Tinto Sin ...,Vino Rosado Sin Alcohol,6.86
3,Carl Jung Merlot – Vino Tinto Sin Alcohol,Vino Rosado Sin Alcohol,6.86
4,Schloss Raggendorf Zweigelt – Vino Tinto Sin A...,Vino Rosado Sin Alcohol,6.86


### 1.1.2

Obtenemos datos especificios sobre cada producto de la pagina inicial

Paso 1: Conseguir las URL de cada producto

In [462]:
lista_urls = []

In [463]:
lista_urls = []
for div in sopa.find_all('div', {'class':'product-wrap'}): 
    url = div.find('a').attrs['href']
    lista_urls.append(url)

Paso 2. Defino las características que describen cada botella de vino y posteriormente lo aplico a cada URL

In [490]:
url_producto_indv = "https://www.thebluedolphinstore.com/producto/carl-jung-rose-vino-rosado-sin-alcohol/"

In [491]:
driver = webdriver.Chrome()   # abre una ventana de chrome

driver.get(url_producto_indv)   # entra en la pagina

In [492]:
producto_indv = driver.find_element(By.CLASS_NAME, "woocommerce-product-details__short-description").text.split('\n')

producto_indv

['Vino rosado sin alcohol',
 'Carl Jung Rosé es un delicioso vino rosado sin alcohol que tiene hasta 1/3 menos calorías que un vino rosado normal. Por tanto permite disfrutar de forma más saludable y si estás a dieta, sin perder nada de sabor.',
 'Este vino rosado sin alcohol de Carl Jung tiene todas las ventajas de un rosado típico pero nada de alcohol. Además, esta marca produce vinos sin alcohol veganos.',
 'Notas de Cata',
 'De color rosa sutil, ligeramente brillante, finamente afrutado y refrescante. Acidez armoniosa, taninos sutiles, frutal con recuerdos a frambuesas y fresas.',
 'Volumen: 75 cl.',
 'Variedad:',
 'Graduación: <0,5 %',
 'Productor: Carl Jung (Alemania)']

In [493]:
notas_de_cata = producto_indv[4]
notas_de_cata

'De color rosa sutil, ligeramente brillante, finamente afrutado y refrescante. Acidez armoniosa, taninos sutiles, frutal con recuerdos a frambuesas y fresas.'

In [495]:
for element in producto_indv:
    if element.startswith('Volumen'):
        volumen = int(element.replace('Volumen: ', '').replace('<.','').replace('cl.',''))
    if element.startswith('Variedad'):
        variedad = element.replace('Variedad: ', '')
    if element.startswith('Graduación'): 
        graduacion = element.replace('Graduación: ', '')
    if element.startswith('Productor'): 
        productor = element.replace('Productor: ', '')
    if element.startswith('Productor'):
        pais = element.split('(')[1].replace(')','')
    if element in producto_indv[4]:
        notas_de_cata = notas_de_cata

In [440]:
volumen

75

In [441]:
variedad

'Variedad:'

In [442]:
graduacion

'<0,5 %'

In [443]:
productor

'Carl Jung (Alemania)'

In [444]:
pais

'Alemania'

In [445]:
etiqueta = driver.find_element(By.CLASS_NAME, "tagged_as").text

etiqueta

'Etiqueta: Vegano'

In [446]:
etiqueta = etiqueta.replace('Etiqueta: ', '')
        
etiqueta

'Vegano'

In [520]:
## Tuve que instalar el paquete de selenium porque me daba esta excepcion en la etiqueta
from selenium.common.exceptions import NoSuchElementException

Paso 3: Creo la data2 con cada valor unico y obtenemos el siguiente DataFrame
Este paso me ha dado muchos errores:
1. He tenido que incluir muchos valores de .replace() porque no en todas las url estan definidos de la misma forma, y por ello me cortaba el bucle una y otra vez.
2. Cada pagina tiene que abrir, buscar y cerrarse, por eso agrego los driver.get() y driver.close()
3. Etiqueta me daba muchos errores agregamos la excepcion de NoSuchElementException y le pedimos que cuando esto ocurra nos arroje el valor de No Disponible.
4. Paciencia.

In [517]:
data2 = []

for u in lista_urls:
    driver = webdriver.Chrome()
    driver.get(u) 
   
    producto_indv = driver.find_element(By.CLASS_NAME, "woocommerce-product-details__short-description").text.split('\n')
    
    for element in producto_indv:
        if element.startswith('Volumen'):
            volumen = int(element.replace('Volumen: ', '').replace('75 .','').replace('.','').replace('cl','').replace('cl.',''))
        if element.startswith('Variedad'):
            variedad = element.replace('Variedad: ', '')
        if element.startswith('Graduación'): 
            graduacion = element.replace('Graduación: ', '')
        if element.startswith('Productor'): 
            productor = element.replace('Productor: ', '')
        if element.startswith('Productor'):
            pais = element.split('(')[1].replace(')','')
        if element in producto_indv[4]:
            notas_de_cata = notas_de_cata
    try:
        etiqueta = driver.find_element(By.CLASS_NAME, "tagged_as").text.replace('Etiqueta: ', '')
        etiqueta = etiqueta.replace('Etiqueta: ', '').replace('Etiquetas: ' ,'')
    except NoSuchElementException:
        etiqueta = "No Disponible"

    
    dictio2 = {'volumen' : volumen, 'graduacion': graduacion, 'productor':productor, 'pais': pais, 'etiqueta': etiqueta, 'notas de cata': notas_de_cata,'url': url}
    
    data2.append(dictio2)
    
    driver.close()
    

In [518]:
data2

[{'volumen': 75,
  'graduacion': '<0,5 %',
  'productor': 'Carl Jung (Alemania)',
  'pais': 'Alemania',
  'etiqueta': 'Vegano',
  'notas de cata': 'De color rosa sutil, ligeramente brillante, finamente afrutado y refrescante. Acidez armoniosa, taninos sutiles, frutal con recuerdos a frambuesas y fresas.',
  'url': 'https://www.thebluedolphinstore.com/producto/vino-sin-alcohol-zero-zero-blanco/'},
 {'volumen': 75,
  'graduacion': '0,5%',
  'productor': 'Carl Jung (Alemania)',
  'pais': 'Alemania',
  'etiqueta': 'Syrah',
  'notas de cata': 'De color rosa sutil, ligeramente brillante, finamente afrutado y refrescante. Acidez armoniosa, taninos sutiles, frutal con recuerdos a frambuesas y fresas.',
  'url': 'https://www.thebluedolphinstore.com/producto/vino-sin-alcohol-zero-zero-blanco/'},
 {'volumen': 75,
  'graduacion': '<0,5 %',
  'productor': 'Carl Jung (Alemania)',
  'pais': 'Alemania',
  'etiqueta': 'Cabernet Sauvignon, Vegano',
  'notas de cata': 'De color rosa sutil, ligeramente br

In [528]:
listado_2 = pd.DataFrame(data2)

listado_2.head()

,volumen,graduacion,productor,pais,etiqueta,notas de cata,url
0,75,"<0,5 %",Carl Jung (Alemania),Alemania,Vegano,"De color rosa sutil, ligeramente brillante, fi...",https://www.thebluedolphinstore.com/producto/v...
1,75,"0,5%",Carl Jung (Alemania),Alemania,Syrah,"De color rosa sutil, ligeramente brillante, fi...",https://www.thebluedolphinstore.com/producto/v...
2,75,"<0,5 %",Carl Jung (Alemania),Alemania,"Cabernet Sauvignon, Vegano","De color rosa sutil, ligeramente brillante, fi...",https://www.thebluedolphinstore.com/producto/v...
3,75,"<0,5 %",Carl Jung (Alemania),Alemania,"Merlot, Vegano","De color rosa sutil, ligeramente brillante, fi...",https://www.thebluedolphinstore.com/producto/v...
4,75,"0,5%",Carl Jung (Alemania),Alemania,No Disponible,"De color rosa sutil, ligeramente brillante, fi...",https://www.thebluedolphinstore.com/producto/v...


Paso 3: Procedo a concatenar los dos DataFrame y crear la lista definitiva.

In [532]:
lista_vinos = pd.concat([listado , listado_2], axis = 1)

lista_vinos.head()

,nombre,variedad,precio,volumen,graduacion,productor,pais,etiqueta,notas de cata,url
0,Carl Jung Rosé – Vino Rosado Sin Alcohol,Vino Rosado Sin Alcohol,6.86,75,"<0,5 %",Carl Jung (Alemania),Alemania,Vegano,"De color rosa sutil, ligeramente brillante, fi...",https://www.thebluedolphinstore.com/producto/v...
1,Schloss Raggendorf Shiraz Rosé – Vino Rosado S...,Vino Rosado Sin Alcohol,6.86,75,"0,5%",Carl Jung (Alemania),Alemania,Syrah,"De color rosa sutil, ligeramente brillante, fi...",https://www.thebluedolphinstore.com/producto/v...
2,Carl Jung Cabernet Sauvignon – Vino Tinto Sin ...,Vino Rosado Sin Alcohol,6.86,75,"<0,5 %",Carl Jung (Alemania),Alemania,"Cabernet Sauvignon, Vegano","De color rosa sutil, ligeramente brillante, fi...",https://www.thebluedolphinstore.com/producto/v...
3,Carl Jung Merlot – Vino Tinto Sin Alcohol,Vino Rosado Sin Alcohol,6.86,75,"<0,5 %",Carl Jung (Alemania),Alemania,"Merlot, Vegano","De color rosa sutil, ligeramente brillante, fi...",https://www.thebluedolphinstore.com/producto/v...
4,Schloss Raggendorf Zweigelt – Vino Tinto Sin A...,Vino Rosado Sin Alcohol,6.86,75,"0,5%",Carl Jung (Alemania),Alemania,No Disponible,"De color rosa sutil, ligeramente brillante, fi...",https://www.thebluedolphinstore.com/producto/v...


Paso 4: exporto a CSV la tabla.

In [537]:
lista_vinos.to_csv('lista_vinos', index=False)